# Extraction of ITS1-5.8S rRNA-ITS2 Sequence From RefseqITS Records

- RefSeq ITS records are not perfect.
- Some records contain additional regions like '18S ribosomal RNA' or '28S ribosomal RNA'
- Some records do not contain ITS1 or ITS2 regions. Some records are incomplete.
- Following code extracts ITS1-5.8S rRNA-ITS2 region from GenBank RefSeq ITS records.

In [4]:
# Download gbff format ITS sequence records
! wget https://ftp.ncbi.nlm.nih.gov/refseq/TargetedLoci/Fungi/fungi.ITS.gbff.gz
! gunzip --force fungi.ITS.gbff.gz # --force override existing file 
gbff_file = 'fungi.ITS.gbff'


from Bio import SeqIO
# Parse the GBFF file
records = SeqIO.parse(gbff_file, "genbank")
reference_regions=[]
for record in records:
    ribosomal_rna_present=False
    ITS1_present=False
    ITS2_present=False
    for feature in record.features:
        # Look for "internal transcribed spacer 1" and "internal transcribed spacer 2"
        if feature.type == "misc_RNA" and "product" in feature.qualifiers:
            product_name = feature.qualifiers["product"][0]
            if product_name == "internal transcribed spacer 1":
                start = int(feature.location.start)
                ITS1_present=True
            elif product_name == "internal transcribed spacer 2":
                end = int(feature.location.end)
                ITS2_present=True
        # Look for "5.8S ribosomal RNA"
        elif feature.type == "rRNA" and "product" in feature.qualifiers:
            product_name = feature.qualifiers["product"][0]
            if product_name == "5.8S ribosomal RNA":
                ribosomal_rna_present=True
    # If the record is complete, write the region of interest into a new fasta file. 
    if all([ribosomal_rna_present, ITS1_present,ITS2_present]):
        sequence = record.seq[start:end]
        accession = record.id
        description = record.description
        reference_region = SeqIO.SeqRecord(sequence, id=str(accession), description=str(description))
        reference_regions.append(reference_region)

# Write region of interest
output_file_name = 'reference_regions.fa'
SeqIO.write(reference_regions, output_file_name, "fasta")

--2023-09-24 14:05:41--  https://ftp.ncbi.nlm.nih.gov/refseq/TargetedLoci/Fungi/fungi.ITS.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.229, 130.14.250.13
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8391142 (8.0M) [application/x-gzip]
Saving to: 'fungi.ITS.gbff.gz.1'

fungi.ITS.gbff.gz.1 100%[===================>]   8.00M  3.37MB/s    in 2.4s    

2023-09-24 14:05:44 (3.37 MB/s) - 'fungi.ITS.gbff.gz.1' saved [8391142/8391142]



16562